In [40]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
import join
import csv
import sys
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
print("【訊息】目前的工作目錄:",os.getcwd())

【訊息】目前的工作目錄: D:\Mypython\專題


- cp950 : Big-5, ascii
- utf-8 : utf-8
- utf-8-sig : utf-8 with BOM

In [2]:
csv.field_size_limit(int(sys.maxsize/10000000000))

131072

In [3]:
df=pd.read_csv(r'D:\Mypython\專題\deal_all.csv', sep=',', engine='python', encoding='utf-8-sig')

In [4]:
df.shape

(546216, 42)

In [ ]:
df.columns

In [ ]:
df.dtypes

In [54]:
df.head(n=2)

,business_hour_guid,seller_GUID,business_hour_order_time_s,business_hour_order_time_e,business_hour_deliver_time_s,business_hour_deliver_time_e,pic_alt,zip_code,city_name,district_name,...,develope_sales_id,operation_sales_id,slug,ordered_quantity,ordered_total,category_list,freight_amount,discount_price,is_wms,ancestor_business_hour_guid
0,29B37E53-021C-48E9-909E-C6C151974AC7,EC5DDAD4-F8F8-418B-8AED-7BAC2553F2B6,2000-03-01 00:00:00.000,2000-03-01 01:00:00.000,2000-03-01 00:00:00.000,2000-03-01 00:00:59.000,柯絲妃兒/胎盤素/肌因賦活/身體精華乳,407,台中市,西屯區,...,1113531340,1.113531e+09,NaN,0,0.0,[88|221|124|769|136|139],0.0,NaN,0,NaN
1,2C6E83A9-9785-4593-A759-07A78AB309BB,B6CC8A1E-37C8-44E5-B6E2-15123387AF95,2001-01-01 00:00:00.000,2001-01-01 00:00:00.000,2001-01-01 00:00:00.000,2001-01-01 00:00:59.000,aibo 領導者/ 立體聲/智慧藍牙/耳機麥克風系列,243,新北市,泰山區,...,1112804144,1.112804e+09,NaN,0,0.0,[88|221|117|2179|2180|100105|100106|136|139],0.0,NaN,0,NaN


In [5]:
df.loc[:]['item_name']

0                             柯絲妃兒胎盤素肌因賦活身體精華乳10入
1                    aibo領導者-立體聲智慧藍牙耳機麥克風系列（2入方案）
2                    aibo領導者-立體聲智慧藍牙耳機麥克風系列（4入方案）
3                    aibo領導者-立體聲智慧藍牙耳機麥克風系列（1入方案）
4                    aibo領導者-立體聲智慧藍牙耳機麥克風系列（8入方案）
5                          aibo領導者-立體聲智慧藍牙耳機麥克風系列
6                       【黑橋牌】招牌香腸(原味/蒜味) 任選 15包/組
7                        【黑橋牌】招牌香腸(原味/蒜味) 任選 8包/組
8                        【黑橋牌】招牌香腸(原味/蒜味) 任選 5包/組
9                             【黑橋牌】招牌香腸(原味/蒜味) 任選
10                      【黑橋牌】招牌香腸(原味/蒜味) 任選 10包/組
11                                   水林-台農57號冰心地瓜
12                     THERMOS膳魔師-超輕量不鏽鋼真空保溫杯/保溫瓶
13                                      匯鮮市集-智利鮭魚
14                         自由拳擊有氧健身系列－特厚多功能拳擊沙袋4入
15                         自由拳擊有氧健身系列－直立式拳擊球拳擊座2入
16                         自由拳擊有氧健身系列－特厚多功能拳擊沙袋1入
17                 自由拳擊有氧健身系列 特厚多功能拳擊沙袋－半指拳擊手套組1組
18                   尚順育樂世界-一票暢遊尚順！晴天雨天都可以玩HIGH專案
19                                    尚順育樂世界-門票四張


In [18]:
df['item_name']=np.array(df['item_name'],dtype="str")

In [20]:
name = []
for i in range(0,len(df)):
    if (df.loc[i]['item_name'].find("【")) >= 0 :
        if (''.join(df.loc[i]['item_name'].split("】")[-1]).strip()) == "" :
            name.append(''.join(df.loc[i]['item_name'].split("】")[-2]).strip())
        else :
            name.append(''.join(df.loc[i]['item_name'].split("】")[-1]).strip())
    else :
        name.append(''.join(df.loc[i]['item_name']).strip())

In [23]:
name2 = []
for i in name :
    name2.append(i.replace('=','').replace('-','').
                 replace('(','').replace(')','').
                 replace('+','').replace('【','').
                 replace('/','').replace('\\','').
                 replace('!','').replace('，','').
                 replace('#','').replace('_','').
                 replace(';','').replace(':','').replace(' ',''))

In [32]:
list(jieba.cut(name2[0]))

['柯絲妃', '兒胎', '盤素', '肌因', '賦活', '身體', '精華乳', '10', '入']

In [36]:
name2[546215]

'除塵紙10包200張200片組'

In [25]:
stopword =[]
for i in open(r'D:\Mypython\專題\停用詞.txt') :
    stopword.append(i.rstrip())

In [37]:
nameall = []
for i in range(0,len(name2)):
    l = []
    out =''
    l = jieba.cut(name2[i])
    for j in l:
        if j not in stopword:
            out = out+j
    nameall.append(out)

In [39]:
nameall

['柯絲妃兒胎盤素肌因賦活身體精華乳10入',
 'aibo領導者立體聲智慧藍牙耳機麥克風系列入方案',
 'aibo領導者立體聲智慧藍牙耳機麥克風系列入方案',
 'aibo領導者立體聲智慧藍牙耳機麥克風系列入方案',
 'aibo領導者立體聲智慧藍牙耳機麥克風系列入方案',
 'aibo領導者立體聲智慧藍牙耳機麥克風系列',
 '招牌香腸原味蒜味任選15包組',
 '招牌香腸原味蒜味任選包組',
 '招牌香腸原味蒜味任選包組',
 '招牌香腸原味蒜味任選',
 '招牌香腸原味蒜味任選10包組',
 '水林台農57號冰心地瓜',
 'THERMOS膳魔師超輕量鏽鋼真空保溫杯保溫瓶',
 '匯鮮市集智利鮭魚',
 '自由拳擊有氧健身系列－特厚多功能拳擊沙袋入',
 '自由拳擊有氧健身系列－直立式拳擊球拳擊座入',
 '自由拳擊有氧健身系列－特厚多功能拳擊沙袋入',
 '自由拳擊有氧健身系列特厚多功能拳擊沙袋－半指拳擊手套組組',
 '尚順育樂世界一票暢遊尚順晴天雨天玩HIGH專案',
 '尚順育樂世界門票四張',
 '自由拳擊有氧健身系列直立式拳擊球拳擊座－競技拳擊手套組組',
 'iPhone7iPhone7plus0.28超薄軟殼必買',
 '自由拳擊有氧健身系列－直立式拳擊球拳擊座入',
 '自由拳擊有氧健身系列',
 '自由拳擊有氧健身系列直立式拳擊球拳擊座－半指拳擊手套組組',
 '尚順育樂世界門票二張',
 '自由拳擊有氧健身系列－特厚多功能拳擊沙袋入',
 '自由拳擊有氧健身系列特厚多功能拳擊沙袋－競技拳擊手套組組',
 'OSTER美國BallMasonJar隨鮮瓶果汁機BLSTMM',
 '中國結刺繡紅包袋入',
 '新優植黑木耳露350ml一組30瓶',
 '新優植黑木耳露350ml兩組60瓶',
 '生活新優植黑木耳露350ml',
 '日用日用量多夜任選盒組',
 '日用日用量多夜任選盒組',
 '日用日用量多夜任選盒組',
 '日用日用量多夜任選',
 '日用日用量多夜任選盒組',
 '日用日用量多夜任選盒組',
 '韓版卡通行李箱保護套加厚版XL碼入',
 'CASA萬用黑金爐SGS認證日式無煙烤盤CAF7172入',
 'CASA萬用黑金爐SGS認證日式無煙烤盤CAF7171入',
 'SUPERGA義大利國民經典男女款休閒鞋雙',
 

In [42]:
df['code_name']=np.array(df['code_name'],dtype="str")
name3 = []
for i in range(0,len(df)):
    name3.append(df.loc[i]['code_name'].replace('/',''))

In [49]:
name3

['nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 '即食調理熟食小吃',
 '即食調理熟食小吃',
 '即食調理熟食小吃',
 '即食調理熟食小吃',
 '即食調理熟食小吃',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',

In [52]:
nameall_2 = [] 
for i in range(0,len(nameall)):
    nameall_2.append(nameall[i]+name3[i])
nameall_2 = np.array(nameall_2,dtype="str")

In [53]:
nameall_2

array(['柯絲妃兒胎盤素肌因賦活身體精華乳10入nan', 'aibo領導者立體聲智慧藍牙耳機麥克風系列入方案nan',
       'aibo領導者立體聲智慧藍牙耳機麥克風系列入方案nan', ..., '迷你除塵掃地機器附除塵紙80張入組掃地機器人',
       '除塵紙30包600張600片組掃地機器人', '除塵紙10包200張200片組掃地機器人'], dtype='<U63')

In [68]:
pd.DataFrame(nameall_2).to_csv(r"D:\MyPython\專題\總和.csv", encoding='utf_8_sig')

### 關鍵字抽取

In [87]:
df2=pd.read_csv(r"D:\MyPython\專題\總和.csv", sep=',', engine='python')

In [88]:
df2.head(2)

,name
0,柯絲妃兒胎盤素肌因賦活身體精華乳10入nan
1,aibo領導者立體聲智慧藍牙耳機麥克風系列入方案nan


In [89]:
df2.columns

Index(['name'], dtype='object')

In [74]:
def word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [91]:
df2['name']=np.array(df2['name'],dtype="str")

In [92]:
df2["content_cutted"] = df2['name'].apply(word_cut)

In [93]:
df2["content_cutted"]

0                       柯絲妃 兒胎 盤素 肌因 賦活 身體 精華乳 10 入 nan
1                  aibo 領導者 立體聲 智慧 藍牙耳機 麥克風 系列 入 方案 nan
2                  aibo 領導者 立體聲 智慧 藍牙耳機 麥克風 系列 入 方案 nan
3                  aibo 領導者 立體聲 智慧 藍牙耳機 麥克風 系列 入 方案 nan
4                  aibo 領導者 立體聲 智慧 藍牙耳機 麥克風 系列 入 方案 nan
5                       aibo 領導者 立體聲 智慧 藍牙耳機 麥克風 系列 nan
6                      招牌 香腸 原味 蒜味 任選 15 包組 即食 調理 熟食 小吃
7                         招牌 香腸 原味 蒜味 任選 包組 即食 調理 熟食 小吃
8                         招牌 香腸 原味 蒜味 任選 包組 即食 調理 熟食 小吃
9                            招牌 香腸 原味 蒜味 任選 即食 調理 熟食 小吃
10                     招牌 香腸 原味 蒜味 任選 10 包組 即食 調理 熟食 小吃
11                                 水林 台農 57 號 冰心 地瓜 nan
12                THERMOS 膳 魔師 超輕 量 鏽 鋼 真空 保溫杯 保溫 瓶 nan
13                                      匯鮮 市集 智利 鮭魚 nan
14                  自由 拳擊 有氧 健身 系列 － 特厚 多功能 拳擊 沙袋 入 nan
15                   自由 拳擊 有氧 健身 系列 － 直立式 拳擊球 拳擊 座入 nan
16                  自由 拳擊 有氧 健身 系列 － 特厚 多功能 拳擊 沙袋 入 nan
17        自由 拳擊 有氧 健身 系列 特厚 多功能 拳擊 沙袋 － 半指 拳擊 手套

In [97]:
n_features = 6000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.95,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df2.content_cutted)

In [98]:
tf_feature_names = tf_vectorizer.get_feature_names()
print(len(tf_feature_names))
tf_feature_names

6000


['003',
 '02',
 '10',
 '100',
 '1000',
 '10000mah',
 '1000g',
 '1000ml',
 '100g',
 '100ml',
 '108',
 '1080p',
 '10cm',
 '10g',
 '10kg',
 '10l',
 '10ml',
 '10nan',
 '10row',
 '10w',
 '10wled',
 '11',
 '110',
 '115',
 '116',
 '118',
 '11cm',
 '12',
 '120',
 '12000mah',
 '12000plus',
 '120g',
 '123',
 '125',
 '125g',
 '128',
 '129',
 '12l',
 '13',
 '130',
 '13alife',
 '14',
 '140',
 '144',
 '15',
 '150',
 '15000mah',
 '150g',
 '150ml',
 '15g',
 '16',
 '160',
 '160g',
 '16g',
 '16gb',
 '16gb1',
 '16l',
 '17',
 '17life',
 '17mall',
 '18',
 '180',
 '18000mah',
 '180d',
 '180g',
 '18650',
 '19',
 '192',
 '1a',
 '1beauty',
 '1cm',
 '1kg',
 '1l',
 '1m',
 '20',
 '200',
 '2000',
 '20000',
 '20000mah',
 '200g',
 '200ml',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '20cm',
 '20g',
 '20l',
 '21',
 '216',
 '22',
 '220',
 '220g',
 '23',
 '24',
 '240',
 '24cm',
 '24h',
 '24k',
 '25',
 '250',
 '250g',
 '250ml',
 '25g',
 '26',
 '26g',
 '27',
 '270',
 '28',
 '280',
 '280d',
 '288',
 '2

In [114]:
outf=open(file=r'D:\MyPython\專題\out_3.txt',mode="wt")

In [115]:
for i in tf_feature_names[4001:6001]:
    outf.write(i+"\n")
outf.close()

### pyspark
- https://decisionstats.com/2017/08/31/importing-data-from-csv-file-using-pyspark/amp/

In [121]:
from pyspark import SparkContext, SparkConf

In [122]:
sc = SparkContext()

In [123]:
df2 = sc.textFile(r'D:\Mypython\專題\deal_all.csv')

In [124]:
df2.count()

546217

In [125]:
type(df2)

pyspark.rdd.RDD

In [127]:
df2.take(5)

['business_hour_guid,seller_GUID,business_hour_order_time_s,business_hour_order_time_e,business_hour_deliver_time_s,business_hour_deliver_time_e,pic_alt,zip_code,city_name,district_name,item_name,item_orig_price,item_price,item_default_daily_amount,event_name,event_title,product_spec,shopping_cart,combo_pack_count,city_list,deal_acc_business_group_id,acc_business_group_name,deal_type_detail,code_name,code,is_travel_deal,installment_3months,installment_6months,installment_12months,deny_installment,shippingdate,is_long_contract,develope_sales_id,operation_sales_id,slug,ordered_quantity,ordered_total,category_list,freight_amount,discount_price,is_wms,ancestor_business_hour_guid',
 '29B37E53-021C-48E9-909E-C6C151974AC7,EC5DDAD4-F8F8-418B-8AED-7BAC2553F2B6,2000-03-01 00:00:00.000,2000-03-01 01:00:00.000,2000-03-01 00:00:00.000,2000-03-01 00:00:59.000,柯絲妃兒/胎盤素/肌因賦活/身體精華乳,407,台中市,西屯區,柯絲妃兒胎盤素肌因賦活身體精華乳10入,8800.00,1490.00,20,。,,,0,1,[477|492],7,宅配,NULL,NULL,NULL,0,1,0,0,0,0,0,1113531340,11135313